In [1]:
import os
CLIENT_ID = os.environ.get('CLIENT_ID')
CLIENT_SECRET = os.environ.get('CLIENT_SECRET')
VERSION = '20180605'

In [2]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v='20180323',
ll='40.7243,-74.0018',
query='hospital',
limit=1
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

data

{'meta': {'code': 200, 'requestId': '5f2d60121cb0e54c1e34219f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'warning': {'text': 'There aren\'t a lot of results for "hospital." Try something more general, reset your filters, or expand the search area.'},
  'suggestedRadius': 600,
  'headerLocation': 'SoHo',
  'headerFullLocation': 'SoHo, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'hospital',
  'totalResults': 28,
  'suggestedBounds': {'ne': {'lat': 40.730159829892735,
    'lng': -74.00578832439199},
   'sw': {'lat': 40.727460170107285, 'lng': -74.00307167560801}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e99bebc2c5bf14846cd48c3',
       'name': 'VillageCare Rehab

In [4]:
from pandas.io.json import json_normalize

In [6]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
venues = data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-7-2b8baa831883>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,VillageCare Rehabilitation & Nursing Center,Hospital,40.72881,-74.00443


In [21]:
from geopy.geocoders import Nominatim


address = 'Toronto, CA'
geolocator = Nominatim(user_agent='toronto_hospitals')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


In [22]:
import json, requests
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v='20180323',
ll=f'{latitude}, {longitude}',
query='hospital',
limit=20
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

data

{'meta': {'code': 200, 'requestId': '5f2d6352f7e86f7793e9cd44'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'query': 'hospital',
  'totalResults': 31,
  'suggestedBounds': {'ne': {'lat': 43.659057188352406,
    'lng': -79.37570457440934},
   'sw': {'lat': 43.65256705210062, 'lng': -79.3919815037503}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ad4c064f964a5206ff820e3',
       'name': "St. Michael's Hospital",
       'contact': {},
       'location': {'address': '30 Bond St',
        'crossStreet': 'at Queen St E',
      

In [23]:
venues = data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-23-619f89ca2758>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,St. Michael's Hospital,Hospital,43.653784,-79.377809
1,Dr. Archie Chung & Associates,Hospital,43.653649,-79.379904
2,The Hospital for Sick Children (SickKids),Hospital,43.657499,-79.386512
3,Sleep Lab,Hospital,43.656671,-79.386320
4,St Michael Hospital,Hospital,43.654024,-79.378736
5,St Michael's Hospital - Gastroenterology,Hospital,43.653653,-79.378081
6,St. Michael's Maternity Ward,Hospital,43.653089,-79.377908
7,Step By Step Professional Family Foot Care,Hospital,43.652862,-79.377736
8,Diagnostic Imaging,Hospital,43.656543,-79.388633
9,St Michaels Martin Family Centre,Hospital,43.653105,-79.377439


In [24]:
address = 'New York, USA'
geolocator = Nominatim(user_agent='newyork_hospitals')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

40.7127281 -74.0060152


In [25]:
url = 'https://api.foursquare.com/v2/venues/explore'

params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v='20180323',
ll=f'{latitude}, {longitude}',
query='hospital',
limit=20
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

data

{'meta': {'code': 200, 'requestId': '5f2d732e2002af58d5f5c9f2'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'Downtown Manhattan',
  'headerFullLocation': 'Downtown Manhattan, New York',
  'headerLocationGranularity': 'neighborhood',
  'query': 'hospital',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 40.72275791520996,
    'lng': -73.99428123235703},
   'sw': {'lat': 40.703568765942386, 'lng': -74.01552563905716}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5c741911b9a5a8002c14ffd1',
       'name': 'CityMD Tribeca Urgent Care - NYC',
       'contact': {},
       'location': {'address': '87 Chambers St',
        'lat': 40.714875,
      

In [26]:
venues = data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-26-619f89ca2758>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,CityMD Tribeca Urgent Care - NYC,Hospital,40.714875,-74.007059
1,Downtown Medical Association,Hospital,40.710618,-74.004595
2,NewYork-Presbyterian-Lower Manhattan Hospital,Hospital,40.709917,-74.004758
3,CityMD Fulton,Hospital,40.710265,-74.008340
4,Lynn Flynn,Hospital,40.711776,-74.010451
5,NYC Department of Health & Mental Hygiene,Hospital,40.715812,-74.002887
6,One Medical,Hospital,40.716762,-74.009468
7,"Vanguard Dermatology - New York, NY - Financia...",Hospital,40.708685,-74.011549
8,FACE-Aesthetic Training Centers,Hospital,40.718098,-74.002434
9,"Advanced Dermatology, P.C.",Hospital,40.718500,-74.009286


In [27]:
address = 'Washington, USA'
geolocator = Nominatim(user_agent='newyork_hospitals')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.8949855 -77.0365708


In [28]:
params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v='20180323',
ll=f'{latitude}, {longitude}',
query='hospital',
limit=20
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

data

{'meta': {'code': 200, 'requestId': '5f2d74916dd9a7231d8b93b4'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 924,
  'headerLocation': 'Northwest Washington',
  'headerFullLocation': 'Northwest Washington, Washington',
  'headerLocationGranularity': 'neighborhood',
  'query': 'hospital',
  'totalResults': 81,
  'suggestedBounds': {'ne': {'lat': 38.90204378706899,
    'lng': -77.03727334141732},
   'sw': {'lat': 38.895162471551274, 'lng': -77.04298419356346}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5ada761bf1fdaf5e1caee9d7',
       'name': 'Providence Health System Gastroenterology',
       'contact': {},
       'location': {'address': '1150 Varnum Street Northeast',
   

In [29]:
venues = data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-29-619f89ca2758>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Providence Health System Gastroenterology,Hospital,38.895642,-77.037817
1,Providence Health System Thoracic Surgery,Hospital,38.895559,-77.037882
2,Providence Health System Cardiology,Hospital,38.895492,-77.038075
3,Providence Health System GYN Group,Hospital,38.895626,-77.037662
4,Providence Health System Center for Otolaryngo...,Hospital,38.895634,-77.037629
5,Providence Health System Diabetes & Nutrition ...,Hospital,38.895759,-77.037855
6,Providence Health System General Surgery,Hospital,38.895492,-77.037533
7,Providence Health System Sleep Institute,Hospital,38.895759,-77.037683
8,Providence Health System Providence Internal M...,Hospital,38.895567,-77.037801
9,Providence Health System Bariatric Care Center,Hospital,38.895475,-77.037796


In [30]:
address = 'Paris, Fr'
geolocator = Nominatim(user_agent='newyork_hospitals')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

48.8566969 2.3514616


In [35]:
params = dict(
client_id=CLIENT_ID,
client_secret=CLIENT_SECRET,
v='20180323',
ll=f'{latitude}, {longitude}',
query='College',
limit=100
)
resp = requests.get(url=url, params=params)
data = json.loads(resp.text)

data

{'meta': {'code': 200, 'requestId': '5f2d76d388d9f22c7a29de34'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 600,
  'headerLocation': 'Hôtel-de-Ville',
  'headerFullLocation': 'Hôtel-de-Ville, Paris',
  'headerLocationGranularity': 'neighborhood',
  'query': 'college',
  'totalResults': 77,
  'suggestedBounds': {'ne': {'lat': 48.87222625250679,
    'lng': 2.3741414263852727},
   'sw': {'lat': 48.8447742706724, 'lng': 2.3319478760392114}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bda8b322a3a0f477169abb6',
       'name': 'IPESUP',
       'contact': {},
       'location': {'address': '9 rue de Platre',
        'lat': 48.859277260456565,
        'lng': 2.354884293771872,
  

In [36]:
venues = data['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues

<ipython-input-36-619f89ca2758>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,IPESUP,College Academic Building,48.859277,2.354884
1,Centre de Danse du Marais,General College & University,48.859703,2.353626
2,ENM Paris,General College & University,48.853327,2.350334
3,IPESUP,College Academic Building,48.860855,2.353443
4,Prepasup,College Classroom,48.859539,2.355270
...,...,...,...,...
72,RU Mazet,College Cafeteria,48.853654,2.339002
73,Salle de Relaxation,College Gym,48.848586,2.358015
74,Centre Sportif Jean Dame,College Gym,48.866538,2.345246
75,Crous Mabillon,College Cafeteria,48.852513,2.335293
